In [59]:
#from numpy.core.numeric import infty
!pip install numpy
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreceated sklearn

You should consider upgrading via the 'c:\users\friso\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\friso\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\friso\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.
ERROR: Invalid requirement: '#'
You should consider upgrading via the 'c:\users\friso\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Task 1

In [60]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./online_shoppers_intention.csv"))
frame.describe(include='all') # 18 columns, including 2 categorical: Month & VisitorType, and two boolean: Weekend & Revenue

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330,12330.000000,12330.000000,12330.000000,12330.000000,12330,12330,12330
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,3,2,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,May,NaN,NaN,NaN,NaN,Returning_Visitor,False,False
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3364,NaN,NaN,NaN,NaN,10551,9462,10422
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,NaN,2.124006,2.357097,3.147364,4.069586,NaN,NaN,NaN
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,NaN,0.911325,1.717277,2.401591,4.025169,NaN,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,NaN,2.000000,2.000000,1.000000,2.000000,NaN,NaN,NaN
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,NaN,2.000000,2.000000,3.000000,2.000000,NaN,NaN,NaN
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,NaN,3.000000,2.000000,4.000000,4.000000,NaN,NaN,NaN


In [61]:
#visualisation
# split the Browser 13 users from the rest
frame13 = frame.loc[frame.Browser == 13]
frameOther = frame.loc[frame.Browser != 13]

import plotly.graph_objects as go
# make a small dataframe with normalized data for plotting
def create_distribution(frameColumn):
    return pd.DataFrame({
        'Browser 13': frame13[frameColumn].value_counts(normalize=True),
        'Other browsers': frameOther[frameColumn].value_counts(normalize=True) 
    }).reset_index().rename(columns={'index': col}) 
# plot all these columns
for col in ['VisitorType', 'Region', 'Month']:
    distribution = create_distribution(col)
    fig = go.Figure()
    fig.add_trace(go.Bar(x=distribution[col], y=distribution['Browser 13'], name='Browser 13'))
    fig.add_trace(go.Bar(x=distribution[col], y=distribution['Other browsers'], name='Other browsers'))
    fig.update_layout(barmode='group', title=f'{col} Distribution')
    fig.update_yaxes(title='Count')
    fig.update_xaxes(title=f'{col}')
    fig.show()

Task 2

In [62]:
#Prepositioning op basis van algoritmes in task 3
# Preprocess by changing all string types to an integer
def to_num(x):
    data = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
        "May": 5, "June": 6, "Jul": 7, "Aug": 8,
        "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12, 
        "Returning_Visitor": 1, "Old_Visitor": 2, "New_Visitor": 3, "Other": 4, 
        True: 1, False: 0
    }
    if x not in data:
        raise ValueError(f"Invalid month string: {x}")
    return data[x]

preprocessed = (frame.copy().dropna()) # dropna() is added here for a case where the dataset would include missing values, but does nothing with the original dataset as it does not contain any
# converts strings and booleans to numbers
columns = ['Month', 'VisitorType', 'Weekend', 'Revenue']
for column in columns:
    addlist = []
    for i, j in enumerate(preprocessed[column]):
        addlist.append(to_num(j))
    preprocessed[column] = addlist

preprocessed = (preprocessed-preprocessed.min())/(preprocessed.max()-preprocessed.min()) # Normalization
print(preprocessed)


       Administrative  Administrative_Duration  Informational  \
0            0.000000                 0.000000            0.0   
1            0.000000                 0.000000            0.0   
2            0.000000                 0.000000            0.0   
3            0.000000                 0.000000            0.0   
4            0.000000                 0.000000            0.0   
...               ...                      ...            ...   
12325        0.111111                 0.042663            0.0   
12326        0.000000                 0.000000            0.0   
12327        0.000000                 0.000000            0.0   
12328        0.148148                 0.022067            0.0   
12329        0.000000                 0.000000            0.0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0        0.001418                 0.000000   
1                         0.0        0.002837                 0.001000 

Task 3

In [68]:
# affinity propagation clustering - based on documentation - VERY SLOW FOR WHOLE DATASET
from sklearn.cluster import AffinityPropagation, DBSCAN, Birch
import plotly.express as px
Data = preprocessed.copy()
# By sampling it goes faster
Sample = Data.sample(n=1000)
clustering = AffinityPropagation(random_state=2).fit(Sample)
AffinityPropagation(random_state=2)

labels = clustering.labels_
cluster_centers = clustering.cluster_centers_


# DBSCAN clustering
# It works, but a lot of datapoints get labeled -1, meaning there is a lot of noise
# It has less clusters compared to the other methods
clustering2 = DBSCAN().fit(Data)
labels2 = clustering2.labels_

# Birch clustering
clustering3 = Birch(n_clusters=None).fit(Data)
labels3 = clustering3.labels_

fig = px.scatter(Sample, labels)
fig.update_layout(title='Affinity Propagation')
fig.update_xaxes(title='Clusters')
fig.show()
fig = px.scatter(Data, labels2)
fig.update_layout(title='DBSCAN')
fig.update_xaxes(title='Clusters')
fig.show()
fig = px.scatter(Data, labels3)
fig.update_layout(title='Birch')
fig.update_xaxes(title='Clusters')
fig.show()


In [64]:
# Visualisations of task 3

fig = px.scatter(Sample, labels)
fig.update_layout(title='Affinity Propagation')
fig.update_xaxes(title='Clusters')
fig.show()

fig = px.scatter(Data, labels2)
fig.update_layout(title='DBSCAN')
fig.update_xaxes(title='Clusters')
fig.show()

fig = px.scatter(Data, labels3)
fig.update_layout(title='Birch')
fig.update_xaxes(title='Clusters')
fig.show() 

Task 4

In [65]:
import numpy as np


def Silhouette_score(point, labelSELF, labels, Data):
    clusterSELF = Data[labels == labelSELF].to_numpy()
    clusters = Data[labels != labelSELF].to_numpy()
    afstand = 0

    for points in clusterSELF:
        if set(points) == set(point):
            continue
        afstand += np.linalg.norm(point-points)
    a = afstand/len(clusterSELF)

    lowest = 999999999
    for clust in clusters:
        afstand_nearest = 0
        for points in clust:
            afstand_nearest += np.linalg.norm(point-points)
        dis = afstand_nearest/len(clust)
        if dis < lowest:
            lowest = dis
    b = lowest
    return (b-a)/(max(b,a))

def Silhouette_scores(x, labels):
    n = len(x)
    sil_values = []
    for i in range(n):
        sil_values.append(Silhouette_score(point=x.iloc[i].to_numpy(), labelSELF = labels[i], labels = labels, Data= x))
    return(sil_values)

def score_visualizer(labels, silhouette_values):
    fig = go.Figure()

    y_lower = 0
    for cluster in np.unique(labels):
        colors = px.colors.qualitative.Set2

        color = colors[cluster % len(colors)]  # cycle through palette
        cluster_sil = []
        for i in range(len(labels)):
            if labels[i] == cluster:
                cluster_sil.append(silhouette_values[i])
        cluster_sil.sort()
        cluster_size = len(cluster_sil)

        # Add horizontal bar segment for this cluster
        fig.add_trace(go.Bar(
            x=cluster_sil,
            y=list(range(y_lower, y_lower + cluster_size)),
            orientation='h',
            marker_color=color,
            showlegend=False

        ))

        y_lower += cluster_size

    # Add vertical line for average silhouette
    fig.add_vline(x=np.mean(silhouette_values), line_dash="dash", line_color="red")

    # Layout
    fig.update_layout(
        title="Silhouette Plot per Cluster",
        xaxis_title="Silhouette coefficient values",
        yaxis_title="Clustered samples",
        bargap=0.01,
        height=600,
        plot_bgcolor="black",
        paper_bgcolor="black"
    )

    fig.show()

values1 = Silhouette_scores(Sample, labels)
print("33%")
values2 = Silhouette_scores(Data[:1000], labels2[:1000])
print("66%")
values3 = Silhouette_scores(Data[:1000], labels3[:1000])
print("100%")
print()
print("in graph:")

score_visualizer(labels, values1)
score_visualizer(labels2[:1000], values2)
score_visualizer(labels3[:1000], values3)

# Davies Buildin Score
from sklearn.metrics import davies_bouldin_score
print("Davies Bouldin score:")
print(f"Affinity propogation: {davies_bouldin_score(Sample, labels)}")
print(f"DBSCAN: {davies_bouldin_score(Data, labels2)}")
print(f"Birch Clustering: {davies_bouldin_score(Data, labels3)}")
print()
# calinski-harabasz index
from sklearn.metrics import calinski_harabasz_score
print("Calinski Harabasz score:")
print(f"Affinity propogation: {calinski_harabasz_score(Sample, labels)}")
print(f"DBSCAN: {calinski_harabasz_score(Data, labels2)}")
print(f"Birch Clustering: {calinski_harabasz_score(Data, labels3)}")

33%
66%
100%

in graph:


Davies Bouldin score:
Affinity propogation: 1.3829140533245634
DBSCAN: 1.5539969551171195
Birch Clustering: 1.2292758210083246

Calinski Harabasz score:
Affinity propogation: 112.87570114815355
DBSCAN: 722.4646699140505
Birch Clustering: 1044.782194159584


Task 5


In [69]:
# Euclidean distance
def our_euclidean_distance(a, b):
    distance = 0
    for index, _ in enumerate(a):
        distance += (a[index] - b[index]) ** 2
    distance = distance**0.5
    return distance
# Manhattan distance
def our_manhattan_distance(a, b):
    distance = 0
    for index, _ in enumerate(a):
        distance += ((a[index] - b[index]) ** 2) ** 0.5
    return distance    
# Cosine similarity
def our_cosine_distance(a, b):
    numerator = 0
    denominator_a = 0
    denominator_b = 0
    for index, _ in enumerate(a):
        numerator += a[index] * b[index]
        denominator_a += a[index] ** 2
        denominator_b += b[index] ** 2
    similarity = numerator / (denominator_a ** 0.5 * denominator_b ** 0.5)
    return 1 - similarity

# Effect of distances on dbscan
clustering_c1 = DBSCAN(metric=our_euclidean_distance).fit(Data[:1000])
label_c1 = clustering_c1.labels_

clustering_c2 = DBSCAN(eps=0.8, metric=our_manhattan_distance).fit(Data[:1000])
label_c2 = clustering_c2.labels_

clustering_c3 = DBSCAN(eps=0.01, metric=our_cosine_distance).fit(Data[:1000])
label_c3 = clustering_c3.labels_

In [67]:
# Visualisation
import plotly.express as px
fig = px.scatter(Data[:1000], label_c1)
fig.update_layout(title='Euclidean Distance (eps=0.5)')
fig.update_xaxes(title='Clusters')
fig.update_yaxes(title='Count')
fig.show()
fig = px.scatter(Data[:1000], label_c2)
fig.update_layout(title='Manhattan Distance (eps=0.8)')
fig.update_xaxes(title='Clusters')
fig.update_yaxes(title='Count')
fig.show()
fig = px.scatter(Data[:1000], label_c3)
fig.update_layout(title='Cosine Distance (eps=0.01)')
fig.update_xaxes(title='Clusters')
fig.update_yaxes(title='Count')
fig.show()